# Базовые операции с Apache Zookeeper

`Apache Zookeeper` является системой для координации распределенных систем. `Apache Zookeeper` можно рассматривать как базу данных, в которой по CAP теореме выбирается C - consistency и P - partition tolerance. Запрос на запись данных объявляется успешным, только если данные были записаны на кворум узлов кластера. Данные внутри базы данных хранятся в формате ключ значение, при этом записи (znode) могут быть вложенные в другие записи, образуя иерархию, что роднит Apache Zookeeper с обычной unix-подобной файловой системой.

In [ ]:
echo "$ZOOKEEPER_HOST":"$ZOOKEEPER_PORT"

## Команда `config`

Команда `config` позволяет посмотреть конфигурация Zookeeper кластера (ensemble)

In [ ]:
zookeeper-shell "$ZOOKEEPER_HOST":"$ZOOKEEPER_PORT" config

## Команда `ls`

Команда `ls` аналогична команде `ls` при работе с файловой системой в Unix и позволяет получить список записей (znode) в определенном пути.

### Посмотреть узлы (`znode`) в корне

Опция `-s` позволяет посмотреть статистику по узлу:

In [ ]:
zookeeper-shell "$ZOOKEEPER_HOST":"$ZOOKEEPER_PORT" ls -s /

### Рекурсный просмотр

Опция `-R` позволяет рекурсивно пройти по всем вложенным записям:

In [ ]:
zookeeper-shell "$ZOOKEEPER_HOST":"$ZOOKEEPER_PORT" ls -R /brokers/ids

## Команда get

Команда `get` позволяет получить содержимое записи (znode):

In [ ]:
zookeeper-shell "$ZOOKEEPER_HOST":"$ZOOKEEPER_PORT" get /brokers/ids/1 | tail -n 1 | json_pp

In [ ]:
zookeeper-shell "$ZOOKEEPER_HOST":"$ZOOKEEPER_PORT" get /brokers/ids/2 | tail -n 1 | json_pp

In [ ]:
zookeeper-shell "$ZOOKEEPER_HOST":"$ZOOKEEPER_PORT" get /brokers/ids/3 | tail -n 1 | json_pp

## Команда `create`

Команда `create` позволяет создать новый узел (znode) в `Apache Zookeeper`. Важно отметить, что все предшествующие узлы должны быть созданы:

In [ ]:
zookeeper-shell "$ZOOKEEPER_HOST":"$ZOOKEEPER_PORT" create /current_date `date "+%Y-%m-%d"`

In [ ]:
zookeeper-shell "$ZOOKEEPER_HOST":"$ZOOKEEPER_PORT" get /current_date

## Команда `set`

In [ ]:
zookeeper-shell "$ZOOKEEPER_HOST":"$ZOOKEEPER_PORT" set /current_date `date "+%s"`

In [ ]:
zookeeper-shell "$ZOOKEEPER_HOST":"$ZOOKEEPER_PORT" get /current_date

## Команда `delete`

Команда `delete` позволяет удалить узел:

In [ ]:
zookeeper-shell "$ZOOKEEPER_HOST":"$ZOOKEEPER_PORT" delete /current_date

## Команда `sync`

Команда `sync` позволяет принудительно запустить синхронизацию между лидером и репликами:

In [ ]:
zookeeper-shell "$ZOOKEEPER_HOST":"$ZOOKEEPER_PORT" sync /

## Задание

Добавить в Apache Zookeeper информацию о фильме "Титаник" (1997) в двух форматах:

1. `json`,
1. каждое поле в отдельном узле.

Схема данных:
```
| общая информация
|---- название
|---- год выхода
| создатели
|---- режиссер
|---- главная мужская роль
|---- главная женская роль
| успех
|---- заработок
|-------- общие сборы по миру (в долларах)
|-------- продажи на DVD (в долларах)
| ---- рейтинг
| -------- IMDB
| -------- Кинопоиск
| ---- награды
| -------- Оскар (количество побед)
| -------- Британская киноакадеми (количество побед)
```
Необходимо сохранить иерархическую структуру информации

# Базовые операции с Kafka

## Создать топик

При создании топика необходимо уделять внимание двум вещам:
1. partitions - количество партиций в топике. Apache Kafka гарантирует порядок только в рамках одной партиций. Чем больше партиций, тем больше консьюмеров можно держать в одной консьюмер группе, а значит тем выше параллелизм и пропускная способность (throughput);
1. replication factor (RF) - сколько копий одной партиции будет хранится в кластере.

Базовые рекомендации:

- RF == 3, т.е. 3 копии (реплики) одних и тех же данных будет хранится в кластере. Значение 3 выведено экспериментальным путем и позволяет обеспечеить высокую надежность даже на очень больших кластерах
- partitions = K * N, где N - ожидаемое количество консьюмеров в консьюмер группе, а K (>= 1) - целочисленный коэффициент. Так количество партиций будет кратным количеству консьюмеров, а значит консьюмеры будут равномерно нагружены.

In [ ]:
kafka-topics --bootstrap-server kafka1:9092,kafka2:9092,kafka3:9092 \
    --topic movies \
    --create \
    --partitions 2 \
    --replication-factor 3

Количество партиций в топике может быть больше, чем количество брокеров:

In [ ]:
kafka-topics --bootstrap-server kafka1:9092,kafka2:9092,kafka3:9092 \
    --topic movies_too_many_partitions \
    --create \
    --partitions 4 \
    --replication-factor 3

Количество реплик партиции не может превышать количества брокеров:

In [ ]:
kafka-topics --bootstrap-server kafka1:9092,kafka2:9092,kafka3:9092, \
    --topic movies_too_many_replicas \
    --create \
    --partitions 4 \
    --replication-factor 4 || true

При создании топика можно указать дополнительные опции при помощи ключа `--config`:

In [ ]:
kafka-topics --bootstrap-server kafka1:9092,kafka2:9092,kafka3:9092 \
    --topic movies_cleanup_policy \
    --create \
    --partitions 4 \
    --replication-factor 3 \
    --config cleanup.policy=compact

Ниже приведен полный список дополнительных параметров:

In [ ]:
kafka-topics --help |& sed -n '/^\s*--config/,/--bootstrap-server/p'

## Получить список топиков

In [ ]:
kafka-topics --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
    --list

In [ ]:
zookeeper-shell "$ZOOKEEPER_HOST":"$ZOOKEEPER_PORT" \
    ls -R /brokers/topics

## Получить информацию о топике

In [ ]:
kafka-topics --bootstrap-server kafka1:9092,kafka2:9092,kafka3:9092 \
    --describe \
    --topic movies

In [ ]:
zookeeper-shell "$ZOOKEEPER_HOST":"$ZOOKEEPER_PORT" \
    get /brokers/topics/movies \
    | tail -n 1 | json_pp

In [ ]:
zookeeper-shell zoo2:2181 \
    get /brokers/topics/movies/partitions/0/state \
    | tail -n 1 | json_pp

In [ ]:
zookeeper-shell zoo2:2181 \
    get /brokers/topics/movies/partitions/1/state \
    | tail -n 1 | json_pp

## Записать данные в топик

Утилита `kafka-console-producer` позволяет записать в топик данные со стандартного потока ввода. При этом все, что попадает в стандартный поток ввода определяется как значение, и это зачение целиком без ключа отправляется в kafka кластер. Если существует ключ, то необходимо указать дополнительные опции при запуске `kafka-console-producer`:

In [ ]:
kafka-console-producer \
  --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
  --topic movies \
  --property "parse.key=true" \
  --property "key.separator=:" \
<<EOF
0:The Shawshank Redemption, 1994
0:God Father, 1972
0:The Dark Knight, 2008
0:The Godfather Part II, 1974
0:12 Angry Men, 1957
1:Schindler's List, 1993
1:The Lord of the Rings: The Return of the King, 2003
1:Pulp Fiction, 1994
1:The Lord of the Rings: The Fellowship of the Ring, 2001
1:The Good, the Bad and the Ugly, 1966
EOF

## Прочитать данные из топика

Утилита `kafka-console-consumer` позволяет прочитать данные из топика и отправить результат на стандартный поток вывода. При этом по умолчанию печаются только значения, которые находились в топике. Если необходимо напечатать так же ключ и партицию, в которой находилась запись, то необходимо указать дополнительные опции при старте `kafka-console-consumer`:

In [ ]:
kafka-console-consumer \
    --bootstrap-server "$KAFKA_HOST":"$KAFKA_PORT" \
    --topic movies \
    --from-beginning \
    --timeout-ms 10000 \
    --property print.key=true \
    --property print.partition=true

Обратите внимание, что сначала были отображены все записи одной партиции, а потом все записи второй партиции

## Консьюмер группы

Каждый топик имеет одну и более партиций, при этом каждую партицию можно читать параллельно, увеличивая таким образом пропускную способность приложения.

При параллельном чтение данных из топика, невозможно гарантировать FIFO порядок в рамках всего топика, Apache Kafka гарантирует порядок записей только в рамках одной партиции, поэтому необходимо вдумчиво подходить к разбиению топика на партиции. На практике редко бывает необходимо поддерживать FIFO порядок в пределах всего топика, но если такая ситуация возникает, то можно создать топик в котором будет всего лишь одна партиция. При этом сама партция может иметь RF (replication factor) больше единицы для повышения оказоустойчивости.

Для параллельного чтения данных из топика необходимо запустить несколько инстанцев консьюмера, объединив их всех в одну консьюмер группу (consumer group). Для этих целей используется опция `group.id`: у всех консьюмеров в одной консьюмер группе должно быть одно и тоже же значение `group.id`

Откройте [kafka-consumer-1.ipynb](./kafka-consumer-1.ipynb) и запустите консьюмера. Он автоматически завершится через 60 секунд

Откройте [kafka-consumer-2.ipynb](./kafka-consumer-2.ipynb) и запустите консьюмера. Он автоматически завершится через 60 секунд

## Запустить продюссера для двух консьюмеров

**ВНИМАНИЕ** убедитесь, что оба консьюмера из [kafka-consumer-1.ipynb](./kafka-consumer-1.ipynb) и [kafka-consumer-2.ipynb](./kafka-consumer-2.ipynb) запущены

In [ ]:
kafka-consumer-groups --bootstrap-server kafka1:9092,kafka2:9092 --list | grep -q consumer-group-1 &&
kafka-consumer-groups --bootstrap-server kafka1:9092,kafka2:9092 --group consumer-group-1 --describe |
  tail -n 2 |
  awk '{print $7}' |
  sort -u |
  wc -l |
  grep -q "^2$" || {
    echo "Пожалуйста, запустите консьюмеры" &&
    false
  }

Если записать в топик новые данные, которые попадут в разные партиции:

In [ ]:
kafka-console-producer \
  --bootstrap-server kafka1:9092 \
  --topic movies \
  --property "parse.key=true" \
  --property "key.separator=:" \
<<EOF
0:Операция "Ы" и другие приключения Шурика, 1965
0:Место встречи изменить нельзя, 1979
0:Семнадцать мнгновений весны, 1973
0:Собачье сердце, 1988
0:Джельтенмены удачи, 1971
1:Тот самый Мюнхаузен, 1979
1:Служебный роман, 1977
1:Бриллиантовая рука, 1969
1:Кавказская пленница, 1967
1:12 стульев,1971
EOF

Можно заметить, что одна партиция была прочитана одним консьюмером, а вторая партиция была прочитана вторым консьюмером (необходимо переключиться на ноутбуки `kafka-consumer-1.ipynb` и `kafka-consumer-2.ipynb`)

## Задание

1. Создать новый топик `music`;
1. При помощи сайта [birthdayjams.com](https://www.birthdayjams.com/) найти песни, которые были популярный в день вашего рождения в периоде с 0 до 15 лет;
1. Отправить названия песен в топик `music`, при этом ключом записи будет год песни, например: *1999: Christina Aguilera, Genie In A Bottle*;
1. Убедиться, что песни уходят в разные партиции топика `music`;
1. **(*) ДОПОЛНИТЕЛЬНО**: запустить 2 или 3 консьюмера в одной консьюмер группе, чтобы получать данные в разных консьюмерах;
1. **(*) ДОПОЛНИТЕЛЬНО**: повысить количество партиций топика `music` до 5 (удалить топик и создать снова) и посмотреть в какие партиции какие записи будут направлены.